# 1 - Create handle to workspace

In [ ]:
# create handle to workspace

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION="3b57d2fe-08b1-4fe9-b535-f5c4387b9a66"
RESOURCE_GROUP="azureml-rg"
WS_NAME="azureml-ws25"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [ ]:
# Verify that the handle works correctly
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location,":", ws.resource_group)

# 2 - Register the model

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

mlflow_model = Model(
    path="./deploy/credit_defaults_model/",
    type=AssetTypes.MLFLOW_MODEL,
    name="credit_defaults_model",
    description="MLflow Model created from local files.",
)

# Register the model
ml_client.models.create_or_update(mlflow_model)

In [ ]:
registered_model_name = "credit_defaults_model"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

print(latest_model_version)

# 3 - Endpoints and deployments

In [ ]:
# Endpoint:

# An endpoint, in this context, is an HTTPS path that provides an interface for clients to send requests (input data)
# to a trained model and receive the inferencing (scoring) results from the model.


# An endpoint provides:
# Authentication using "key or token" based auth
# TLS(SSL) termination
# A stable scoring URI (endpoint-name.region.inference.ml.azure.com)

In [ ]:
# Deployment

# A deployment is a set of resources required for hosting the model that does the actual inferencing.
# A single endpoint can contain multiple deployments.

# Endpoints and deployments are independent Azure Resource Manager resources that appear in the Azure portal.

# 4 - Create an online endpoint

In [ ]:
import uuid

# Create a unique name for the endpoint
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# define an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
    },
)

In [ ]:
# create the online endpoint
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

# 5 - Deploy model to endpoint

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment

# Choose the latest version of the registered model for deployment
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# define an online deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

In [ ]:
# create the online deployment

blue_deployment = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
# return an object that contains metadata for the endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# print a selection of the endpoint's metadata
print(
    f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}"
)

In [ ]:
# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

# 6 - Test endpoint with sample data

In [ ]:
import os

# Create a directory to store the sample request file.
deploy_dir = "./data"
os.makedirs(deploy_dir, exist_ok=True)

In [ ]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
            ]
                }
}

In [ ]:
# test the blue deployment with the sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="./data/sample-request.json",
)

In [ ]:
logs = ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)
print(logs)